In [1]:
from omg.datamodule import DataModule
from omg.datamodule.dataloader import OMGTorchDataset
from torch_geometric.loader import DataLoader

In [2]:
import torch

In [3]:
from omg.model.model_utils import SinusoidalTimeEmbeddings

In [4]:
dataset = DataModule.from_lmdb(["./../data/mp_20/train.lmdb"])

example_data = OMGTorchDataset(dataset, convert_to_fractional=True)

loader = DataLoader(example_data, batch_size=10)

2024-09-26 23:25:39.480 | INFO     | omg.datamodule.datamodule:add_from_lmdb:832 - Reusing existing LMDB file: dataset_274518dcfedeeb30399261097eb4fcb7.lmdb


In [5]:
for batch in loader:
    t = torch.rand(1)
    print(t,batch)
    break   

tensor([0.0220]) OMGDataBatch(
  n_atoms=[10],
  batch=[90],
  species=[90],
  cell=[10, 3, 3],
  pos=[90, 3],
  property={},
  ptr=[11]
)


In [6]:
prop = torch.rand(10)
print(prop)

tensor([0.0541, 0.3423, 0.7843, 0.5486, 0.4327, 0.4043, 0.8633, 0.0907, 0.7883,
        0.9316])


In [7]:
prop.shape

torch.Size([10])

In [8]:
prop.double()

tensor([0.0541, 0.3423, 0.7843, 0.5486, 0.4327, 0.4043, 0.8633, 0.0907, 0.7883,
        0.9316], dtype=torch.float64)

In [9]:
prop_embd = SinusoidalTimeEmbeddings(64)

In [10]:
prop = prop_embd(prop)

In [11]:
prop.shape

torch.Size([10, 64])

In [12]:
t = t.repeat(10)
t.dtype

torch.float32

In [13]:
t = prop_embd(t)

In [14]:
batch

OMGDataBatch(
  n_atoms=[10],
  batch=[90],
  species=[90],
  cell=[10, 3, 3],
  pos=[90, 3],
  property={},
  ptr=[11]
)

In [15]:
batch.cell.shape

torch.Size([10, 3, 3])

In [16]:
#batch.pos = torch.matmul(batch.pos, torch.inverse(batch.cell))

In [17]:
from omg.model.encoders.cspnet_full import CSPNetFull

/home/pawanprakash/miniconda3/envs/fermat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
model = CSPNetFull(latent_dim=64)
model = model.double()

In [19]:
prop.shape

torch.Size([10, 64])

In [20]:
t.shape

torch.Size([10, 64])

In [21]:
prop = prop.double()


In [22]:
prop.dtype

torch.float64

In [23]:
t.shape

torch.Size([10, 64])

In [24]:
md = model(batch,t = t, prop = prop)

property is here
property is here
property is here
property is here
torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
here


In [25]:
md


Data(species_b=[90, 100], species_eta=[90, 100], pos_b=[90, 3], pos_eta=[90, 3], cell_b=[10, 3, 3], cell_eta=[10, 3, 3])

In [26]:
batch.species

tensor([11, 11, 11, 25, 27, 28,  8,  8,  8,  8,  8,  8, 60, 13, 13, 13, 13, 13,
        13, 13, 13, 29, 29, 29, 29, 48, 49, 49, 49, 52, 52, 52, 52, 33, 44, 45,
        45, 45, 45, 23, 23, 23, 23,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
         9, 70, 50, 56, 56, 48, 55, 37, 37, 69, 17, 17, 17, 17, 17, 17, 67, 14,
        28, 28, 28, 28, 19, 19, 37, 69, 23, 23,  8,  8,  8,  8,  8,  8,  8,  8])

In [27]:
md.species_b

tensor([[ 0.9571,  0.2665, -0.5067,  ..., -0.1364, -0.4413,  0.7678],
        [ 0.9574,  0.2665, -0.5085,  ..., -0.1375, -0.4390,  0.7693],
        [ 0.9549,  0.2665, -0.5055,  ..., -0.1408, -0.4406,  0.7709],
        ...,
        [ 0.0632, -0.7896, -0.7506,  ...,  0.9538, -0.2810,  0.3995],
        [ 0.0659, -0.7951, -0.7557,  ...,  0.9535, -0.2768,  0.3885],
        [ 0.0619, -0.7853, -0.7495,  ...,  0.9571, -0.2772,  0.3990]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)